In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 6.6 MB/s 
     |████████████████████████████████| 181 kB 71.3 MB/s 
     |████████████████████████████████| 144 kB 68.2 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=149eb324f96f7bf44f1c5d6eaaacecf83a64ec815247705443fd4cf2ed7fa4fa
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
# Building a model
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.optimizers as Optimizer
import wandb
from wandb.keras import WandbCallback
import pandas as pd
import numpy as np
import cv2 as cv

from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import plotly.express as px

In [65]:
from tensorflow.keras.utils import to_categorical, normalize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras import layers 
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [75]:
def wandb_init(project,name):
    # 1. Start a new run
    run = wandb.init(
                project=project, 
                entity='elsaravana', 
                name=name,
                config={
                    "activation_1": "relu",
                    "activation_2":"sigmoid",
                    "loss": "binary_crossentropy",
                    "metric": "accuracy",
                    "epoch": 10,
                    "batch_size": 32
                })
    if run is None:
        raise ValueError("Wandb didn't initialize properly")

In [71]:
def vgg16_pretrain_cnn_model(img_size):
    # img_size = 224
    # VGG-16 Architecture
    base_model = VGG16(input_shape = (img_size, img_size, 3), # Shape of our images
    include_top = False, # Leave out the last fully connected layer
    weights = 'imagenet')

    for layer in base_model.layers:
        layer.trainable = False

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(base_model.output)

    # Add a fully connected layer with 512 hidden units and ReLU activation
    x = layers.Dense(512, activation=config.activation_1)(x)

    # Add a dropout rate of 0.5
    x = layers.Dropout(0.5)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(1, activation=config.activation_2)(x)

    model = tf.keras.models.Model(base_model.input, x)
    model.compile(optimizer = Optimizer.Adam(learning_rate=0.0001), loss=config.loss,metrics=[config.metric])
    
    model.summary()
    return model

In [72]:
def resnet50_pretrain_cnn_model(img_size):
  # img_size = 224
  base_model = Sequential()
  base_model.add(ResNet50(input_shape=(img_size, img_size,3), include_top=False, weights='imagenet', pooling='max'))
  base_model.add(Dense(1, activation=config.activation_2))
  base_model.compile(optimizer=Optimizer.Adam(lr=0.0001),loss=config.loss,metrics=[config.metric])
  base_model.summary()
  return base_model

In [73]:
def inceptionv3_pretrain_cnn_model(img_size):
  # img_size = 150
  base_model = InceptionV3(input_shape = (img_size, img_size, 3), include_top = False, weights = 'imagenet')

  for layer in base_model.layers:
    layer.trainable = False

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(base_model.output)

  # Add a fully connected layer with 1024 hidden units and ReLU activation
  x = layers.Dense(1024, activation=config.activation_1)(x)

  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x)

  # Add a final sigmoid layer with 1 node for classification output
  x = layers.Dense(1, activation=config.activation_2)(x)

  model = tf.keras.models.Model(base_model.input, x)
  model.compile(optimizer = Optimizer.Adam(learning_rate=0.0001), loss=config.loss,metrics=[config.metric])
  
  model.summary()
  return model

In [74]:
def alexnet_arch_model(img_size):
  # img_size = 224
  # Initialize the model
  model = Sequential()

  # layer 1: convolutional layer + max-pooling layer
  model.add(Conv2D(filters = 96, kernel_size = (11,11), strides= 4, padding = 'valid', activation='relu', input_shape = (img_size,img_size,3)))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layer 2: convolutional layer + max-pooling layer 
  model.add(Conv2D(filters = 256, kernel_size = (5,5), padding = 'same', activation = config.activation_1))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layers 3-5: three convolutional layers + 1 max-pooling layer
  model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = config.activation_1))
  model.add(Conv2D(filters = 384, kernel_size = (3,3), padding = 'same', activation = config.activation_1))
  model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = 'same', activation = config.activation_1))
  model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

  # layers 6 - 8: two fully connected hidden layers and one fully connected output layer
  model.add(Flatten())
  model.add(Dense(4096, activation = config.activation_1))
  model.add(Dropout(0.5))
  model.add(Dense(4096, activation = config.activation_1))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation = config.activation_2))

  # compile the model with a loss funciton, a metric and and optimizer method for estimating the loss function
  model.compile(optimizer=Optimizer.Adam(lr=0.0001),loss=config.loss,metrics=[config.metric])

  model.summary()
  return model


In [78]:
def pre_process(img_size):
    # Obtain the path 
    dataset_path = "/content/gdrive/MyDrive/Region/Southeast/Georgia/Atlanta/Dr Sexton/"

    # 0: knee
    # 1: shoulder

    img_path = []
    label = []

    for subdir, dirs, files in os.walk(dataset_path):
      for file in files:
        if file.endswith(".jpeg") or file.endswith(".png"):
          image_path = os.path.join(subdir, file)
          img_path.append(image_path)
        if file.endswith(".json"):
          json_file = os.path.join(subdir, file)
          if "copy" not in file: 
            df = pd.read_json(json_file, orient='index')
            if df[0]['Scope_type'] == "knee":
              label.append(0)
            elif df[0]['Scope_type'] == "shoulder":
              label.append(1)
            else:
              continue
    
    col_names =  ['img_path', 'label']
    df  = pd.DataFrame(columns = col_names)
    df['img_path'] = img_path
    df['label'] = label

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # y_train = to_categorical(train_df['label'].values)
    # y_test = to_categorical(test_df['label'].values)

    y_train = train_df['label'].values
    y_test = test_df['label'].values

    X_train = []
    X_test = []

    for img_path in train_df['img_path']:
        image = cv.imread(img_path)
        image = cv.resize(image,(img_size,img_size))
        X_train.append(image)

    for img_path in test_df['img_path']:
        image = cv.imread(img_path)
        image = cv.resize(image,(img_size,img_size))
        X_test.append(image)

    X_train = np.array(X_train).reshape(-1,img_size,img_size,3)
    X_test = np.array(X_test).reshape(-1,img_size,img_size,3)

    X_train = normalize(X_train, axis=1)
    X_test = normalize(X_test, axis=1)

    
    return X_train, y_train, X_test, y_test

In [69]:
def metrics(test_y, predictions):
    accuracy = sklearn.metrics.accuracy_score(test_y, predictions)
    precision = sklearn.metrics.precision_score(test_y, predictions)
    recall = sklearn.metrics.recall_score(test_y, predictions)
    f1 = sklearn.metrics.f1_score(test_y, predictions)
    
    return accuracy, precision, recall, f1

In [68]:
def show_roc_and_pr_curve(test_y, probas,a,p,r,f1):
    fpr,tpr,thr = roc_curve(test_y, probas)
    precision, recall, thresholds = precision_recall_curve(test_y, probas)

    roc_fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f}), Accuracy={a:.2f}',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    roc_fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    roc_fig.update_yaxes(scaleanchor="x", scaleratio=1)
    roc_fig.update_xaxes(constrain='domain')
    roc_fig.show()
    
    pr_fig = px.area(
        x=recall, y=precision,
        title=f'Precision-Recall Curve (AUC={auc(fpr, tpr):.4f}), Precision={p:.2f}, Recall={r:.2f}, F1={f1:.2f}',
        labels=dict(x='Recall', y='Precision'),
        width=700, height=500
    )
    pr_fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0
    )
    pr_fig.update_yaxes(scaleanchor="x", scaleratio=1)
    pr_fig.update_xaxes(constrain='domain')
    pr_fig.show()

## Inception V3 Pretrain model

In [79]:
img_size = 150

wandb_init('algovera-arthroscopic surgery-image-classification',"10_inceptionv3_pretrain_classifier_"+str(img_size)+"_x_"+str(img_size))
config = wandb.config

X_train, y_train, X_test, y_test = pre_process(img_size)

# # Our vectorized labels
# y_train = np.asarray(train_labels).astype('float32').reshape((-1,1))
# y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))

model = inceptionv3_pretrain_cnn_model(img_size)

model.fit(X_train, y_train, epochs = config.epoch, batch_size=config.batch_size, validation_split=0.1, callbacks=[WandbCallback()])
model.save("trained_models/inceptionv3_pretrain_10_epochs_"+str(img_size)+"_x_"+str(img_size))


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 74, 74, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 74, 74, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                            

In [80]:
preds = model.predict(X_test)
flatten_preds_probas = preds.flatten()
actual_preds = (flatten_preds_probas > 0.5).astype("int32")

accuracy, precision, recall, f1_score = metrics(y_test,actual_preds)
print("Accuracy: ", accuracy, "Precision: ", precision, "Recall: ", recall, "F1_score: ", f1_score)
show_roc_and_pr_curve(y_test, flatten_preds_probas,accuracy,precision,recall,f1_score)
wandb.log({"confusion_matrix" : wandb.plot.confusion_matrix(probs=None, preds=actual_preds, y_true=y_test, class_names=["knee","shoulder"])})

Accuracy:  0.75 Precision:  1.0 Recall:  0.5 F1_score:  0.6666666666666666


## VGG-16 Pretrain model

In [83]:
img_size = 224
wandb_init('algovera-arthroscopic surgery-image-classification',"10_vgg16_pretrain_classifier_"+str(img_size)+"_x_"+str(img_size))
config = wandb.config

X_train, y_train, X_test, y_test = pre_process(img_size)

model = vgg16_pretrain_cnn_model(img_size)

model.fit(X_train, y_train, epochs = config.epoch, batch_size=config.batch_size, validation_split=0.1, callbacks=[WandbCallback()])
# model.save("trained_models/vgg16_pretrain_10_epochs_"+str(img_size)+"_x_"+str(img_size))

preds = model.predict(X_test)
flatten_preds_probas = preds.flatten()
actual_preds = (flatten_preds_probas > 0.5).astype("int32")

accuracy, precision, recall, f1_score = metrics(y_test,actual_preds)
print("Accuracy: ", accuracy, "Precision: ", precision, "Recall: ", recall, "F1_score: ", f1_score)
show_roc_and_pr_curve(y_test, flatten_preds_probas,accuracy,precision,recall,f1_score)
wandb.log({"confusion_matrix" : wandb.plot.confusion_matrix(probs=None, preds=actual_preds, y_true=y_test, class_names=["knee","shoulder"])})

accuracy,▁▁▂█▇█▇▇▅█
epoch,▁▂▃▃▄▅▆▆▇█
loss,███▆▆▆▅▄▃▁
val_accuracy,▁█▁▁▁▁▁▁▁▁
val_loss,██▇▆▄▃▃▃▃▁
accuracy,0.78571
best_epoch,9
best_val_loss,0.69162
epoch,9
loss,0.67896
val_accuracy,0.5


58900480/58889256 [==============================] - 0s 0us/step
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

## ResNet 50 Pretrain model

In [81]:
img_size = 224

wandb_init('algovera-arthroscopic surgery-image-classification',"10_resnet50_pretrain_classifier_"+str(img_size)+"_x_"+str(img_size))
config = wandb.config

X_train, y_train, X_test, y_test = pre_process(img_size)

model = resnet50_pretrain_cnn_model(img_size)

model.fit(X_train, y_train, epochs = config.epoch, batch_size=config.batch_size, validation_split=0.1, callbacks=[WandbCallback()])
# model.save("trained_models/resnet50_pretrain_10_epochs_"+str(img_size)+"_x_"+str(img_size))

preds = model.predict(X_test)
flatten_preds_probas = preds.flatten()
actual_preds = (flatten_preds_probas > 0.5).astype("int32")

accuracy, precision, recall, f1_score = metrics(y_test,actual_preds)
print("Accuracy: ", accuracy, "Precision: ", precision, "Recall: ", recall, "F1_score: ", f1_score)
show_roc_and_pr_curve(y_test, flatten_preds_probas,accuracy,precision,recall,f1_score)
wandb.log({"confusion_matrix" : wandb.plot.confusion_matrix(probs=None, preds=actual_preds, y_true=y_test, class_names=["knee","shoulder"])})

accuracy,▁▃▃█▆▄████
epoch,▁▂▃▃▄▅▆▆▇█
loss,▂█▃▁▂▂▁▁▁▁
val_accuracy,▁▁█▁▁█████
val_loss,█▅▁▂▂▁▁▁▁▁
accuracy,1.0
best_epoch,7
best_val_loss,4e-05
epoch,9
loss,0.00068
val_accuracy,1.0


94781440/94765736 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_4 (Dense)             (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 23,536,641
Non-trainable params: 53,120
_________________________________________________________________
Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



1/1 [==============================] - 12s 12s/step - loss: 6.1797 - accuracy: 0.5000 - val_loss: 0.7992 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 327ms/step - loss: 1.2887e-06 - accuracy: 1.0000 - val_loss: 0.8211 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 224ms/step - loss: 7.6316e-07 - accuracy: 1.0000 - val_loss: 0.8435 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 227ms/step - loss: 9.6013e-07 - accuracy: 1.0000 - val_loss: 0.8074 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 1.0882e-06 - accuracy: 1.0000 - val_loss: 0.7410 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 1.7132e-06 - accuracy: 1.0000 - val_loss: 0.6998 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 194ms/step - loss: 3.3776e-06 - accuracy: 1.0000 - val_loss: 0.7179 - val_accuracy: 0.5000
Epoch 8/10
1/1

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



## CNN model with AlexNet Architecture

In [82]:
img_size = 224

wandb_init('algovera-arthroscopic surgery-image-classification',"10_cnn_alexarch_classifier_"+str(img_size)+"_x_"+str(img_size))
config = wandb.config

X_train, y_train, X_test, y_test = pre_process(img_size)

model = alexnet_arch_model(img_size)

model.fit(X_train, y_train, epochs = config.epoch, batch_size=config.batch_size, validation_split=0.1, callbacks=[WandbCallback()])
# model.save("trained_models/cnn_alexarch_10_epochs_"+str(img_size)+"_x_"+str(img_size))

preds = model.predict(X_test)
flatten_preds_probas = preds.flatten()
actual_preds = (flatten_preds_probas > 0.5).astype("int32")

accuracy, precision, recall, f1_score = metrics(y_test,actual_preds)
print("Accuracy: ", accuracy, "Precision: ", precision, "Recall: ", recall, "F1_score: ", f1_score)
show_roc_and_pr_curve(y_test, flatten_preds_probas,accuracy,precision,recall,f1_score)
wandb.log({"confusion_matrix" : wandb.plot.confusion_matrix(probs=None, preds=actual_preds, y_true=y_test, class_names=["knee","shoulder"])})

accuracy,▁█████████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,▆▆█▆▃▁▂▄▆█
accuracy,1.0
best_epoch,5
best_val_loss,0.69979
epoch,9
loss,3e-05
val_accuracy,0.5


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_188 (Conv2D)         (None, 54, 54, 96)        34944     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 26, 26, 96)       0         
 2D)                                                             
                                                                 
 conv2d_189 (Conv2D)         (None, 26, 26, 256)       614656    
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 12, 12, 256)      0         
 2D)                                                             
                                                                 
 conv2d_190 (Conv2D)         (None, 12, 12, 384)       885120    
                                                                 
 conv2d_191 (Conv2D)         (None, 12, 12, 384)      

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



1/1 [==============================] - 5s 5s/step - loss: 0.6929 - accuracy: 0.4286 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 0.6929 - accuracy: 0.4286 - val_loss: 0.6926 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 0.6925 - accuracy: 0.5000 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 0.6891 - accuracy: 0.7857 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 0.6893 - accuracy: 0.7143 - val_loss: 0.6920 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 0.6892 - accuracy: 0.7857 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 72ms/step - loss: 0.6867 - accuracy: 0.7143 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 8/10
1/1 [==============================] - 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

